# Download Dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("prateekagnihotri/cms-trigger-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cms-trigger-dataset


# Import Dependencies

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy.matlib
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam # Changed from *
from tensorflow import keras
from collections import Counter
import tensorflow.keras.backend as K
# from livelossplot import PlotLossesKerasTF # Will remove for cleaner output unless you need it for each
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import f1_score, accuracy_score, roc_curve, auc
from sklearn.metrics import mean_absolute_error as mae_sklearn # Renamed to avoid conflict
from tqdm import tqdm
import os

os.environ['TF_FORCE_GPU_ALLOW_GROWTH_TRUE'] = 'true'
tf.config.set_soft_device_placement(True)
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print('Num GPUs available: '+ str(len(tf.config.experimental.list_physical_devices('GPU'))))
print('TF version: '+str(tf.__version__))

Num GPUs available: 1
TF version: 2.18.0


# Read Dataset

In [ ]:
df = pd.read_csv('/kaggle/input/cms-trigger-dataset/CMS_trigger.csv')

# Preprocess

In [ ]:
variables = np.array(df.loc[:,'Phi_0':'MedianTheta'])
parameters = np.array(df.loc[:,'q/pt':])
q_pt = parameters[:,0]
pT = abs(1/q_pt) # GeV
print('Variables: '+str(variables.shape))
print('Parameters: '+str(parameters.shape))
print('q/pt: '+str(q_pt.shape))
print('pT: '+str(pT.shape)) # Changed from PeV to pT for clarity

scaler = StandardScaler()
data = scaler.fit_transform(variables)

x_train, x_test, pT_tr, pT_ts = train_test_split(data, pT, test_size = 0.2, random_state = 1) # Added pT_tr, pT_ts

print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print("pT_tr shape:", pT_tr.shape)
print("pT_ts shape:", pT_ts.shape)

# For CNN and Multi models
data_2d = np.zeros((len(data),7,4))
data_2d[:,:,0] = np.array(df.loc[:,'Phi_0':'Mask_0':4])
data_2d[:,:,1] = np.array(df.loc[:,'Phi_2':'Mask_2':4])
data_2d[:,:,2] = np.array(df.loc[:,'Phi_3':'Mask_3':4])
data_2d[:,:,3] = np.array(df.loc[:,'Phi_4':'Mask_4':4])
print("data_2d shape:", data_2d.shape)

road = np.array(df.loc[:,'PatternStraightness':'MedianTheta'])
rem_feats = road.copy()
print("rem_feats shape:", rem_feats.shape)

# Split 2D data and remaining features
x_train2, x_test2, rem_tr, rem_ts, pT_tr_2d, pT_ts_2d = train_test_split(
    data_2d, rem_feats, pT, test_size = 0.2, random_state = 1
)
x_train2 = np.expand_dims(x_train2,-1)
x_test2 = np.expand_dims(x_test2,-1)

print("x_train2 shape:", x_train2.shape)
print("x_test2 shape:", x_test2.shape)
print("rem_tr shape:", rem_tr.shape)
print("rem_ts shape:", rem_ts.shape)
print("pT_tr_2d shape:", pT_tr_2d.shape) # This will be same as pT_tr
print("pT_ts_2d shape:", pT_ts_2d.shape) # This will be same as pT_ts

# Callbacks
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", restore_best_weights=True, patience = 10, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=5,verbose=1)

Variables: (1179356, 31)
Parameters: (1179356, 3)
q/pt: (1179356,)
pT: (1179356,)
x_train shape: (943484, 31)
x_test shape: (235872, 31)
pT_tr shape: (943484,)
pT_ts shape: (235872,)
data_2d shape: (1179356, 7, 4)
rem_feats shape: (1179356, 3)
x_train2 shape: (943484, 7, 4, 1)
x_test2 shape: (235872, 7, 4, 1)
rem_tr shape: (943484, 3)
rem_ts shape: (235872, 3)
pT_tr_2d shape: (943484,)
pT_ts_2d shape: (235872,)


# FCNN

In [ ]:
def FCNN(num_class,activation,opt,loss_fn,in_shape,metric):

    I = Input(shape=in_shape)
    x = Dense(512,activation='relu')(I)
    x = Dense(256,activation='relu')(x)
    x = Dense(128,activation='relu')(x)
    x = Dense(128,activation='relu')(x)
    x = Dense(128,activation='relu')(x)
    O = Dense(num_class,activation=activation)(x)

    model = keras.Model(inputs=I, outputs=O)
    model.compile(optimizer = opt, loss = loss_fn)
    return model

In [ ]:
def CNN(num_class,activation_fn,opt,loss_fn,metric):

    activation = tf.keras.layers.LeakyReLU(0.15)
    I = Input(shape=(x_train2.shape[1],x_train2.shape[2],x_train2.shape[3]))
    x = Conv2D(64, kernel_size=(2, 2),padding = 'same',activation=activation)(I)
    x = Conv2D(128, kernel_size=(2, 2),padding = 'same',activation=activation)(x)
    x = MaxPooling2D((2,1))(x)
    x = Conv2D(256, kernel_size=(2, 2),padding = 'same',activation=activation)(x)
    x = Conv2D(256, kernel_size=(2, 2),padding = 'same',activation=activation)(x)
    x = Conv2D(128, kernel_size=(2, 2),padding = 'same',activation=activation)(x)
    x = Conv2D(128, kernel_size=(2, 2),padding = 'same',activation=activation)(x)
#     x = MaxPooling2D((2,2))(x)
#     x = Conv2D(128, kernel_size=(2, 2),padding = 'same',activation=activation)(x)
    x = Flatten()(x)
    x = Dense(256, activation=activation)(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation=activation)(x)
    x = Dropout(0.5)(x)
#     x = Dense(128, activation=activation)(x)
#     x = Dense(64, activation=activation)(x)

    O = Dense(num_class,activation=activation_fn)(x)

    model = keras.Model(inputs=I, outputs=O)
    model.compile(optimizer = opt, loss = loss_fn)

    return model

In [ ]:
results_summary = {}
EPOCHS = 50
BATCH_SIZE = 2**12

print("\n--- Training FCNN ---")
opt_fcnn = Adam()
model_fcnn = FCNN_mse(num_class=1,
                      activation_output='linear',
                      opt=opt_fcnn,
                      loss_fn='mean_squared_error',
                      in_shape=(x_train.shape[1],),
                      metrics_list=['mae'])
print(model_fcnn.summary())
history_fcnn = model_fcnn.fit(x_train, pT_tr,
                              validation_data=(x_test, pT_ts),
                              batch_size=BATCH_SIZE,
                              verbose=1,
                              epochs=EPOCHS,
                              callbacks=[early_stopping, reduce_lr])
loss_fcnn, mae_fcnn = model_fcnn.evaluate(x_test, pT_ts, verbose=0)
print(f"FCNN Test MSE: {loss_fcnn}, Test MAE: {mae_fcnn}")
results_summary["FCNN"] = {"params": model_fcnn.count_params(), "test_mse": loss_fcnn, "test_mae": mae_fcnn}


--- Training FCNN ---


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)          │ (None, 31)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 512)                 │          16,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 213,761 (835.00 KB)

 Trainable params: 213,761 (835.00 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 18000.3223 - mae: 23.5932 - val_loss: 18647.6836 - val_mae: 19.3611 - learning_rate: 0.0010
Epoch 2/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 16802.9062 - mae: 20.5488 - val_loss: 18536.5664 - val_mae: 21.2141 - learning_rate: 0.0010
Epoch 3/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 16565.5352 - mae: 20.1568 - val_loss: 18440.3945 - val_mae: 20.1762 - learning_rate: 0.0010
Epoch 4/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 17520.9551 - mae: 20.2948 - val_loss: 18458.9414 - val_mae: 20.9897 - learning_rate: 0.0010
Epoch 5/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 17717.6133 - mae: 20.1883 - val_loss: 18449.3281 - val_mae: 19.0801 - learning_rate: 0.0010
Epoch 6/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 15987.5371 - mae: 19.5768 - val_loss: 18464.1953 - val_mae: 18.8644 - learning_rate: 0.0010
Epoch 7/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 15722.7705 - mae: 19.4

# CNN-Grid

In [ ]:
def CNN_grid(num_class,activation_fn,opt,loss_fn,metric, num_conv, num_fcn):

    activation = tf.keras.layers.LeakyReLU(0.15)
    I = Input(shape=(x_train2.shape[1],x_train2.shape[2],x_train2.shape[3]))
    x = Conv2D(64, kernel_size=(2, 2),padding = 'same',activation=activation)(I)
    x = Conv2D(128, kernel_size=(2, 2),padding = 'same',activation=activation)(x)
    x = MaxPooling2D((2,1))(x)
    for i in range(num_conv-1):
        x = Conv2D(128, kernel_size=(2, 2),padding = 'same',activation=activation)(x)
    x = MaxPooling2D((2,2))(x)
    x = Flatten()(x)
    x = Dense(128, activation=activation)(x)
    x = Dropout(0.5)(x)
    for i in range(num_fcn):
        x = Dense(128, activation=activation)(x)
        x = Dropout(0.5)(x)

    O = Dense(num_class,activation=activation_fn)(x)

    model = keras.Model(inputs=I, outputs=O)
    model.compile(optimizer=opt, loss=loss_fn, metrics=metric)

    return model

In [ ]:
print("\n--- Training CNN_Grid ---")
opt_cnn_grid = Adam()

num_conv_for_grid = 3
num_fcn_for_grid = 2

model_cnn_grid_instance = CNN_grid(num_class=1,
                                   activation_fn='linear',
                                   opt=opt_cnn_grid,
                                   loss_fn='mean_squared_error',
                                   metric=['mae'],
                                   num_conv=num_conv_for_grid,
                                   num_fcn=num_fcn_for_grid)

print(model_cnn_grid_instance.summary())
history_cnn_grid = model_cnn_grid_instance.fit(x_train2, pT_tr_2d,
                                         validation_data=(x_test2, pT_ts_2d),
                                         batch_size=BATCH_SIZE,
                                         verbose=1,
                                         epochs=EPOCHS,
                                         callbacks=[early_stopping, reduce_lr])
# This line should now work
loss_cnn_grid, mae_cnn_grid = model_cnn_grid_instance.evaluate(x_test2, pT_ts_2d, verbose=0)
print(f"CNN_Grid (conv={num_conv_for_grid}, fcn={num_fcn_for_grid}) Test MSE: {loss_cnn_grid}, Test MAE: {mae_cnn_grid}")
results_summary[f"CNN_Grid_c"] = {"params": model_cnn_grid_instance.count_params(), "test_mse": loss_cnn_grid, "test_mae": mae_cnn_grid}


--- Training CNN_Grid ---


Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_18 (InputLayer)          │ (None, 7, 4, 1)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_41 (Conv2D)                   │ (None, 7, 4, 64)            │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_42 (Conv2D)                   │ (None, 7, 4, 128)           │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 3, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_43 (Conv2D)                   │ (None, 3, 4, 128)           │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_44 (Conv2D)                   │ (None, 3, 4, 128)           │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 1, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_10 (Flatten)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_41 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_42 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_43 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_44 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 230,593 (900.75 KB)

 Trainable params: 230,593 (900.75 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 19s 54ms/step - loss: 17968.8984 - mae: 21.9848 - val_loss: 18777.4004 - val_mae: 20.6122 - learning_rate: 0.0010
Epoch 2/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - loss: 17171.4297 - mae: 21.2154 - val_loss: 18745.0664 - val_mae: 21.3413 - learning_rate: 0.0010
Epoch 3/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - loss: 16554.0156 - mae: 21.0675 - val_loss: 18740.5078 - val_mae: 19.3199 - learning_rate: 0.0010
Epoch 4/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - loss: 17223.4492 - mae: 20.9370 - val_loss: 18735.7461 - val_mae: 19.6346 - learning_rate: 0.0010
Epoch 5/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - loss: 17864.0039 - mae: 21.4516 - val_loss: 18731.2754 - val_mae: 18.9629 - learning_rate: 0.0010
Epoch 6/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - loss: 18318.6699 - mae: 21.1060 - val_loss: 18429.3125 - val_mae: 23.9540 - learning_rate: 0.0010
Epoch 7/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - loss: 17110.1406 

# CNN

In [ ]:
def CNN(num_class,activation_fn,opt,loss_fn,metric):

    activation = tf.keras.layers.LeakyReLU(0.15)
    I = Input(shape=(x_train2.shape[1],x_train2.shape[2],x_train2.shape[3]))
    x = Conv2D(64, kernel_size=(2, 2),padding = 'same',activation=activation)(I)
    x = Conv2D(128, kernel_size=(2, 2),padding = 'same',activation=activation)(x)
    x = MaxPooling2D((2,1))(x)
    x = Conv2D(256, kernel_size=(2, 2),padding = 'same',activation=activation)(x)
    x = Conv2D(256, kernel_size=(2, 2),padding = 'same',activation=activation)(x)
    x = Conv2D(128, kernel_size=(2, 2),padding = 'same',activation=activation)(x)
    x = Conv2D(128, kernel_size=(2, 2),padding = 'same',activation=activation)(x)
#     x = MaxPooling2D((2,2))(x)
#     x = Conv2D(128, kernel_size=(2, 2),padding = 'same',activation=activation)(x)
    x = Flatten()(x)
    x = Dense(256, activation=activation)(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation=activation)(x)
    x = Dropout(0.5)(x)
#     x = Dense(128, activation=activation)(x)
#     x = Dense(64, activation=activation)(x)

    O = Dense(num_class,activation=activation_fn)(x)

    model = keras.Model(inputs=I, outputs=O)
    model.compile(optimizer = opt, loss = loss_fn)

    return model

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [ ]:
print("\n--- Training CNN ---")
opt_cnn = Adam()

cnn_input_shape = (x_train2.shape[1], x_train2.shape[2], x_train2.shape[3])
model_cnn = CNN_mse(num_class=1,
                    activation_fn_output='linear',
                    opt=opt_cnn,
                    loss_fn='mean_squared_error',
                    metrics_list=['mae'],
                    input_shape_cnn=cnn_input_shape)
print(model_cnn.summary())
history_cnn = model_cnn.fit(x_train2, pT_tr_2d,
                            validation_data=(x_test2, pT_ts_2d),
                            batch_size=BATCH_SIZE,
                            verbose=1,
                            epochs=EPOCHS,
                            callbacks=[early_stopping, reduce_lr])
loss_cnn, mae_cnn = model_cnn.evaluate(x_test2, pT_ts_2d, verbose=0)
print(f"CNN Test MSE: {loss_cnn}, Test MAE: {mae_cnn}")
results_summary["CNN"] = {"params": model_cnn.count_params(), "test_mse": loss_cnn, "test_mae": mae_cnn}


--- Training CNN ---


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)          │ (None, 7, 4, 1)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_24 (Conv2D)                   │ (None, 7, 4, 64)            │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_25 (Conv2D)                   │ (None, 7, 4, 128)           │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 3, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_26 (Conv2D)                   │ (None, 3, 4, 256)           │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_27 (Conv2D)                   │ (None, 3, 4, 256)           │         262,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_28 (Conv2D)                   │ (None, 3, 4, 128)           │         131,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_29 (Conv2D)                   │ (None, 3, 4, 128)           │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ (None, 1536)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 256)                 │         393,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,050,305 (4.01 MB)

 Trainable params: 1,050,305 (4.01 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 35s 122ms/step - loss: 17833.8770 - mae: 22.6855 - val_loss: 18788.1621 - val_mae: 18.7456 - learning_rate: 0.0010
Epoch 2/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 26s 113ms/step - loss: 17612.2363 - mae: 21.7193 - val_loss: 18720.2539 - val_mae: 19.0097 - learning_rate: 0.0010
Epoch 3/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 25s 108ms/step - loss: 16961.4785 - mae: 21.1569 - val_loss: 18646.5566 - val_mae: 22.7500 - learning_rate: 0.0010
Epoch 4/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 25s 107ms/step - loss: 17153.6426 - mae: 20.3847 - val_loss: 17997.7949 - val_mae: 22.7447 - learning_rate: 0.0010
Epoch 5/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 25s 108ms/step - loss: 16031.3867 - mae: 19.0623 - val_loss: 17366.7891 - val_mae: 16.3464 - learning_rate: 0.0010
Epoch 6/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 25s 108ms/step - loss: 15718.0449 - mae: 17.9918 - val_loss: 17167.9980 - val_mae: 15.2745 - learning_rate: 0.0010
Epoch 7/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 25s 108ms/step - loss: 15

# FCNN+CNN

In [ ]:
def multi(num_class,activation_fn,opt,loss_fn,metric):

#     activation = tf.keras.layers.LeakyReLU(0.15)
    activation = 'relu'
    I = Input(shape=(x_train2.shape[1],x_train2.shape[2],x_train2.shape[3]))
    x = Conv2D(32, kernel_size=(2, 2), padding = 'same', activation=activation)(I)
#     x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, kernel_size=(2, 2), padding = 'same', activation=activation)(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, kernel_size=(2, 2), padding = 'same', activation=activation)(x)
#     x = MaxPooling2D((2,2))(x)
    x = Flatten()(x)

    I2 = Input(shape=(rem_tr.shape[1]))
    y = Dense(128, activation=activation)(I2)
    y = Dense(64, activation=activation)(y)
    y = Dense(64, activation=activation)(y)

    z = concatenate([x,y])

    z = Dense(32, activation=activation)(z)

    O = Dense(num_class,activation=activation_fn)(z)

    model = keras.Model(inputs=[I,I2], outputs=O)
    model.compile(optimizer = opt, loss = loss_fn)

    return model

In [ ]:
print("\n--- Training Multi (FCNN+CNN) ---")
opt_multi = Adam()

fcnn_branch_input_shape = rem_tr.shape[1]
model_multi = Multi_mse(num_class=1,
                        activation_fn_output='linear',
                        opt=opt_multi,
                        loss_fn='mean_squared_error',
                        metrics_list=['mae'],
                        input_shape_cnn=cnn_input_shape,
                        input_shape_fcnn_branch=fcnn_branch_input_shape)
print(model_multi.summary())
history_multi = model_multi.fit([x_train2, rem_tr], pT_tr_2d, # Use pT_tr_2d
                                validation_data=([x_test2, rem_ts], pT_ts_2d), # Use pT_ts_2d
                                batch_size=BATCH_SIZE,
                                verbose=1,
                                epochs=EPOCHS,
                                callbacks=[early_stopping, reduce_lr])
loss_multi, mae_multi = model_multi.evaluate([x_test2, rem_ts], pT_ts_2d, verbose=0)
print(f"Multi Test MSE: {loss_multi}, Test MAE: {mae_multi}")
results_summary["Multi"] = {"params": model_multi.count_params(), "test_mse": loss_multi, "test_mae": mae_multi}


--- Training Multi (FCNN+CNN) ---


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14            │ (None, 7, 4, 1)        │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_30 (Conv2D)        │ (None, 7, 4, 32)       │            160 │ input_layer_14[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_31 (Conv2D)        │ (None, 7, 4, 64)       │          8,256 │ conv2d_30[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_15            │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_7           │ (None, 3, 2, 64)       │              0 │ conv2d_31[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_28 (Dense)          │ (None, 128)            │            512 │ input_layer_15[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_32 (Conv2D)        │ (None, 3, 2, 64)       │         16,448 │ max_pooling2d_7[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_29 (Dense)          │ (None, 64)             │          8,256 │ dense_28[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_7 (Flatten)       │ (None, 384)            │              0 │ conv2d_32[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_30 (Dense)          │ (None, 64)             │          4,160 │ dense_29[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 448)            │              0 │ flatten_7[0][0],       │
│ (Concatenate)             │                        │                │ dense_30[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_31 (Dense)          │ (None, 32)             │         14,368 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_32 (Dense)          │ (None, 1)              │             33 │ dense_31[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 52,193 (203.88 KB)

 Trainable params: 52,193 (203.88 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - loss: 16530.5352 - mae: 22.1139 - val_loss: 18771.1055 - val_mae: 21.9292 - learning_rate: 0.0010
Epoch 2/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 17275.5391 - mae: 21.6858 - val_loss: 18752.2031 - val_mae: 19.9487 - learning_rate: 0.0010
Epoch 3/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 17046.2324 - mae: 21.7570 - val_loss: 18677.5137 - val_mae: 22.2140 - learning_rate: 0.0010
Epoch 4/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 17867.6758 - mae: 21.9605 - val_loss: 18554.0781 - val_mae: 20.2679 - learning_rate: 0.0010
Epoch 5/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 17445.9316 - mae: 21.6590 - val_loss: 18302.2910 - val_mae: 18.7905 - learning_rate: 0.0010
Epoch 6/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 16964.6953 - mae: 21.4140 - val_loss: 18009.1289 - val_mae: 19.3230 - learning_rate: 0.0010
Epoch 7/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 17078.4043 - mae

# LSTM

In [ ]:
def LSTM_mse(num_class, activation_fn_output, opt, loss_fn, input_shape_lstm, metrics_list):
    I_lstm = Input(shape=input_shape_lstm) # Expects (timesteps, features)
    x = LSTM(64, return_sequences=True)(I_lstm)
    x = Dropout(0.2)(x)
    x = LSTM(32)(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    O_lstm = Dense(num_class, activation=activation_fn_output)(x)

    model = keras.Model(inputs=I_lstm, outputs=O_lstm)
    model.compile(optimizer=opt, loss=loss_fn, metrics=metrics_list)
    return model

In [ ]:
print("\n--- Training LSTM ---")
x_train2_lstm = np.squeeze(x_train2, axis=-1)
x_test2_lstm = np.squeeze(x_test2, axis=-1)
lstm_input_shape = (x_train2_lstm.shape[1], x_train2_lstm.shape[2])

opt_lstm = Adam()
model_lstm = LSTM_mse(num_class=1,
                      activation_fn_output='linear',
                      opt=opt_lstm,
                      loss_fn='mean_squared_error',
                      input_shape_lstm=lstm_input_shape,
                      metrics_list=['mae'])
print(model_lstm.summary())
history_lstm = model_lstm.fit(x_train2_lstm, pT_tr_2d, # Use pT_tr_2d
                              validation_data=(x_test2_lstm, pT_ts_2d), # Use pT_ts_2d
                              batch_size=BATCH_SIZE,
                              verbose=1,
                              epochs=EPOCHS,
                              callbacks=[early_stopping, reduce_lr])
loss_lstm, mae_lstm = model_lstm.evaluate(x_test2_lstm, pT_ts_2d, verbose=0)
print(f"LSTM Test MSE: {loss_lstm}, Test MAE: {mae_lstm}")
results_summary["LSTM"] = {"params": model_lstm.count_params(), "test_mse": loss_lstm, "test_mae": mae_lstm}


--- Training LSTM ---


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)           │ (None, 7, 4)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 7, 64)               │          17,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 7, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 32,257 (126.00 KB)

 Trainable params: 32,257 (126.00 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 12s 30ms/step - loss: 17666.5684 - mae: 21.5383 - val_loss: 18871.7090 - val_mae: 20.3993 - learning_rate: 0.0010
Epoch 2/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 18136.9688 - mae: 21.9321 - val_loss: 18773.6699 - val_mae: 20.0198 - learning_rate: 0.0010
Epoch 3/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - loss: 17722.4746 - mae: 21.9327 - val_loss: 18666.7363 - val_mae: 21.5802 - learning_rate: 0.0010
Epoch 4/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - loss: 16257.4688 - mae: 20.0372 - val_loss: 17767.6621 - val_mae: 16.7809 - learning_rate: 0.0010
Epoch 5/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - loss: 15982.7217 - mae: 18.5726 - val_loss: 17997.8594 - val_mae: 21.4568 - learning_rate: 0.0010
Epoch 6/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - loss: 17106.0820 - mae: 20.0076 - val_loss: 18674.2168 - val_mae: 23.0107 - learning_rate: 0.0010
Epoch 7/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 16832.5645 - ma